In [ ]:
from docplex.mp.model import Model
import pandas as pd

# sava datas in a matrix of matrixes
first_data = [[0, 67, 23, 89], [56, 0, 45, 23], [12, 34, 0, 78], [78, 90, 12, 0]]
second_data = [[0, 12, 7, 15], [8, 0, 14, 3], [17, 6, 0, 11], [9, 13, 5, 0]]
third_data = [[0, 78, 12, 34], [23, 0, 67, 45], [89, 23, 0, 56], [45, 78, 90, 0]]

data = [first_data, second_data, third_data]

# parameters
c = 4  # number of nodes
d = [8, 9, 10, 11]  # time intervals
tw = [[8, 12], [8, 12], [8, 12]]  # time windows
v = 2  # number of vehicles
M = 100000  # big M

# define model
mdl = Model("CVRP")

# decision variables
x = mdl.binary_var_cube(c, c, len(d) - 1, name="x")
T = mdl.continuous_var_list(c + 1, name="t")

# nodes constraints
mdl.add_constraint(
    mdl.sum(mdl.sum(x[0, j, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

mdl.add_constraint(
    mdl.sum(mdl.sum(x[j, 0, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(x[i, j, t] for i in range(c) if i != j) for t in range(len(d) - 1)
        )
        == 1
    )

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[i, j, t] for i in range(1, c)) for t in range(len(d) - 1))
        == mdl.sum(mdl.sum(x[j, i, t] for i in range(1, c)) for t in range(len(d) - 1))
    )

# time constraints
# for j in range(1, c):
#     for t in range(len(d) - 1):
#         mdl.add_constraint(
#             (1 - x[0, j, t]) * M + T[j]
#             >= d[t] * 60 * mdl.sum(x[0, i, t] for i in range(c)) + data[t][0][j]
#         )

for i in range( 1,c):
    for j in range(1,c):
        for t in range(len(d) - 1):
            if i != j:
                mdl.add_constraint((1 - x[i, j, t]) * M +T[j] -T[i]>=data[t][i][j])

# for j in range(1, c):
#     for t in range(len(d) - 1):
#         mdl.add_constraint(T[j + 1] <= M * (1 - x[j, 0, t] + data[t][j][0]))

# for i in range(1, c):
#     mdl.add_constraint(tw[i - 1][0] * 60 <= T[i])

# for i in range(1, c):
#     mdl.add_constraint(T[i] <= tw[i - 1][1] * 60)

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(mdl.sum(x[i, j, t] * data[t][i][j] for i in range(c)) for j in range(c))
        for t in range(len(d) - 1)
    )
)

sol = mdl.solve(log_output=True)

if sol is None:
    print("No solution found")

print(sol.objective_value)
print(sol.get_values(T))

# print results
# for i in range(c):
#     for j in range(c):
#         for t in range(len(d) - 1):
#             if x_sol[i, j, t] == 1:
#                 print(f"From {i} to {j} in time interval {t}")

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 19 rows and 44 columns.
Reduced MIP has 7 rows, 9 columns, and 24 nonzeros.
Reduced MIP has 9 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Found incumbent of value 43.000000 after 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 7 rows and 9 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.08 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.08 ticks)
31.0
[0, 0, 0, 0, 0]
